# Main Notebook

## Summary

This notebook will do following steps:
 -  Pull validation data
 -  Load CTR predictions from CSV for X different models
 -  Load BaseBid Predictions from pickels for X different models
 -  For X models combine (in Y different ways) the above two steps to build X*Y BidPrices for each impression
 -  Hold auction for each impression one by one
 -  Summary Scores for each of X*Y models 

# Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns
from score_models import Scorer

%matplotlib inline

# Pull Validation Data

In [2]:
path = "C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/"
#test_df = pd.read_csv(path+'test.csv')
#train_df = pd.read_csv(path+'train.csv')
validation_df = pd.read_csv(path+'validation.csv')

In [3]:
#print(validation_df.head())
#avgCTR =validation_df.loc[validation_df["click"] == 1].shape[0]/(validation_df.shape[0])
#print(avgCTR)

# Load CTR Predictions

Need a function that will combine multiple dataframes into one

In [4]:
#path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/click_predictions_achilleas/'
path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/predictions/'

lr_data = pd.read_csv(path+'lr_pCTR.csv')
lr_pCTR=lr_data[['click_proba']]
lr_pCTR = lr_pCTR.rename(index=str, columns={"click_proba": "pCTR"})
print(lr_pCTR.head(2))

xgb_data = pd.read_csv(path+'xgb_pCTR.csv')
xgb_pCTR=xgb_data[['click_proba']]
xgb_pCTR = lr_pCTR.rename(index=str, columns={"click_proba": "pCTR"})
print(lr_pCTR.head(2))



#path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/click_predictions_dwane/'

mF_data = pd.read_csv(path+'click_predictions_narrow.csv')
mF_pCTR=mF_data[['pCTR']]
#mF_pCTR = mF_pCTR.rename(index=str, columns={"click_proba": "pCTR"})
print(mF_pCTR.head(2))






       pCTR
0  0.470948
1  0.393028
       pCTR
0  0.470948
1  0.393028
   pCTR
0   0.0
1   0.0


In [5]:
#resultCTR=teamAdf
#resultCTR["teamB_CTR"]=teamBdf["CTR"]
#result = pd.merge(teamAdf, teamBdf, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
#print(resultCTR.head())


# Load Basebid Predictions

Need a function that will combine multiple dataframes into one

In [6]:
#path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/basebid_predictions_achilleas/'
path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/predictions/'

r_data = pd.read_csv(path+'ridge_basebid.csv')
r_pBB = r_data[['predicted_payprice']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(r_pBB.head(2))

lasso_basebid = pd.read_csv(path+'lasso_basebid.csv')
lasso_pBB = lasso_basebid[['predicted_payprice']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(lasso_pBB.head(2))

elasticnet_basebid = pd.read_csv(path+'elasticnet_basebid.csv')
eN_pBB = elasticnet_basebid[['predicted_payprice']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(eN_pBB.head(2))

rfr_basebid = pd.read_csv(path+'rfg_basebid.csv')
rfr_pBB = rfr_basebid[['predicted_payprice']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(rfr_pBB.head(2))

EightyNineBB=r_pBB * 0 + 89






   predicted_payprice
0           81.330752
1           95.890274
   predicted_payprice
0           78.236533
1           85.978842
   predicted_payprice
0           83.225539
1           85.902317
   predicted_payprice
0           49.996513
1           63.256951


# Formulas to use

In [7]:
def CostantBidding(resBaseBid,resCTR):
    Bids=resBaseBid['predicted_payprice'].values * 0 + 79
    #print("Bids:",Bids)
    return Bids

In [8]:
def RandomBidding(resBaseBid,resCTR):
    Bids = np.random.randint(69, 89+1, resBaseBid.shape[0])
    return Bids

In [9]:
def BestLinearBidding(resBaseBid,resCTR):
    avCTR = resCTR['pCTR'].mean()
    Bids = resBaseBid['predicted_payprice'].values * (resCTR['pCTR'].values/avCTR)
    return Bids

In [10]:
def Strategy1(sub_basebid,sub_pCTR):
    avgCTR = sub_pCTR["pCTR"].mean()
    base_bid = 105.46
    bidprice = (sub_pCTR["pCTR"] * base_bid) / avgCTR
    return bidprice

In [11]:
def Strategy2(sub_basebid,sub_pCTR):
    avgCTR = sub_pCTR["pCTR"].mean()
    bidprice = (sub_pCTR["pCTR"].values * sub_basebid["predicted_payprice"].values) / avgCTR
    return bidprice

In [12]:
# def Function1(resBaseBid,resCTR):
#     avCTR = 0.5
#     Bids=resBaseBid['base_bid'].values * (resCTR['pCTR'].values/avCTR)
#     #print("Bids:",Bids)
#     return Bids

In [13]:
# def Function2(reBaseBid,reCTR):
#     avCTR = 0.5
#     reCTR=reCTR.copy(True)
#     reCTR=(((reCTR+0.5)**2)-0.5)
#     reCTR[reCTR < 0] = 0
#     Bids=reBaseBid.values * (reCTR/avCTR)
#     return Bids

In [14]:
# def Function3(rBaseBid,rCTR):
#     avCTR = 0.5
#     rCTR=rCTR.copy(True)
#     rCTR[rCTR < 0.5] = 0
#     #rCTR=(((rCTR+0.5)**4)-0.5)
#     Bids=rBaseBid.values*1.2 * (rCTR/avCTR)
#     return Bids

# Build Bid Prices for all models

Need a function that will build a new dataframe conatining bids of all teams by combining the previous two dataframes.

In [15]:
resultsBids=pd.DataFrame(index=range(validation_df.shape[0]))
resultsBids['click']=validation_df['click']
resultsBids['payprice']=validation_df['payprice']

resultsBids['ConstantBidding'] = CostantBidding(EightyNineBB,lr_data)
resultsBids['RandomBidding'] = RandomBidding(EightyNineBB,lr_data)
resultsBids['BestLinear'] = BestLinearBidding(EightyNineBB,xgb_pCTR)
resultsBids['xgb_lasso_Str2'] = Strategy2(lasso_pBB,xgb_pCTR)
resultsBids['mF_rfr_Str2'] = Strategy2(rfr_pBB,mF_pCTR)
resultsBids['mF_eN_Str1'] = Strategy1(eN_pBB,mF_pCTR)

multiples = 7
resultsBids['ConstantBidding'] = resultsBids['ConstantBidding'].values * multiples
resultsBids['RandomBidding'] = resultsBids['RandomBidding'].values * multiples
resultsBids['BestLinear'] = resultsBids['BestLinear'].values * multiples
resultsBids['xgb_lasso_Str2'] = resultsBids['xgb_lasso_Str2'].values * multiples
resultsBids['mF_rfr_Str2'] = resultsBids['mF_rfr_Str2'].values * multiples
resultsBids['mF_eN_Str1'] = resultsBids['mF_eN_Str1'].values * multiples

In [17]:
for multiples in range(50,80,5):
    resultsBids['ConstantBidding'] = CostantBidding(EightyNineBB,lr_data)
    resultsBids['RandomBidding'] = RandomBidding(EightyNineBB,lr_data)
    resultsBids['BestLinear'] = BestLinearBidding(EightyNineBB,xgb_pCTR)
    resultsBids['xgb_lasso_Str2'] = Strategy2(lasso_pBB,xgb_pCTR)
    resultsBids['mF_rfr_Str2'] = Strategy2(rfr_pBB,mF_pCTR)
    resultsBids['mF_eN_Str1'] = Strategy1(eN_pBB,mF_pCTR)

    resultsBids['ConstantBidding'] = resultsBids['ConstantBidding'].values * multiples/10
    resultsBids['RandomBidding'] = resultsBids['RandomBidding'].values * multiples/10
    resultsBids['BestLinear'] = resultsBids['BestLinear'].values * multiples/10
    resultsBids['xgb_lasso_Str2'] = resultsBids['xgb_lasso_Str2'].values * multiples/10
    resultsBids['mF_rfr_Str2'] = resultsBids['mF_rfr_Str2'].values * multiples/10
    resultsBids['mF_eN_Str1'] = resultsBids['mF_eN_Str1'].values * multiples/10

    dataSubset = resultsBids.sample(frac=0.2, replace=False).copy(True) #divided by 5 to get similar environment
    dataSubset.reset_index(drop=True, inplace=True)

    s = Scorer()
    df_summary,df = s.set_df(dataSubset, ['ConstantBidding', 'RandomBidding', 'BestLinear','xgb_lasso_Str2', 'mF_rfr_Str2', 'mF_eN_Str1'],6250000,20) 
    # /5 because 6 teams instead of 30
    print("Multiples of budget spent:",multiples/10)
    print(df_summary.sort_values(by=['click']))
    print()
    print()

Multiples of budget spent: 5.0
         budget  click        team_name    win
0  1.261658e+06      4   xgb_lasso_Str2  11855
1  2.154869e+02      2       BestLinear  15222
2  2.079547e+02     14       mF_eN_Str1   4348
3  5.482496e+01     14      mF_rfr_Str2   6625
4  1.025131e+06      1    RandomBidding  13057
5  2.594907e+06      0  ConstantBidding   9678


Multiples of budget spent: 5.5
         budget  click        team_name    win
0  2.092652e+05      6    RandomBidding  13741
1  5.816816e+02     15       mF_eN_Str1   3915
2  4.117345e+02      2       BestLinear  13831
3  4.928700e+05      8   xgb_lasso_Str2  12514
4  4.256847e+02     14      mF_rfr_Str2   6264
5  1.886333e+06      3  ConstantBidding  10520


Multiples of budget spent: 6.0
          budget  click        team_name    win
0     350.753655     11       mF_eN_Str1   3611
1     325.457082      1       BestLinear  12672
2  746198.949527      3  ConstantBidding  13298
3     470.549549      6    RandomBidding  13036
4    

In [19]:
for multiples in range(150,160,5):
    resultsBids['ConstantBidding'] = CostantBidding(EightyNineBB,lr_data)
    resultsBids['RandomBidding'] = RandomBidding(EightyNineBB,lr_data)
    resultsBids['BestLinear'] = BestLinearBidding(EightyNineBB,xgb_pCTR)
    resultsBids['xgb_lasso_Str2'] = Strategy2(lasso_pBB,xgb_pCTR)
    resultsBids['mF_rfr_Str2'] = Strategy2(rfr_pBB,mF_pCTR)
    resultsBids['mF_eN_Str1'] = Strategy1(eN_pBB,mF_pCTR)

    resultsBids['ConstantBidding'] = resultsBids['ConstantBidding'].values * multiples/10
    resultsBids['RandomBidding'] = resultsBids['RandomBidding'].values * multiples/10
    resultsBids['BestLinear'] = resultsBids['BestLinear'].values * multiples/10
    resultsBids['xgb_lasso_Str2'] = resultsBids['xgb_lasso_Str2'].values * multiples/10
    resultsBids['mF_rfr_Str2'] = resultsBids['mF_rfr_Str2'].values * multiples/10
    resultsBids['mF_eN_Str1'] = resultsBids['mF_eN_Str1'].values * multiples/10

    dataSubset = resultsBids.sample(frac=0.2, replace=False).copy(True) #divided by 5 to get similar environment
    dataSubset.reset_index(drop=True, inplace=True)

    s = Scorer()
    df_summary,df = s.set_df(dataSubset, ['ConstantBidding', 'RandomBidding', 'BestLinear','xgb_lasso_Str2', 'mF_rfr_Str2', 'mF_eN_Str1'],6250000,20) 
    # /5 because 6 teams instead of 30
    print("Multiples of budget spent:",multiples/10)
    print(df_summary.sort_values(by=['click']))
    print()
    print()

Multiples of budget spent: 15.0
        budget  click        team_name    win
0   277.185649      0   xgb_lasso_Str2   4984
1   259.659424      0       BestLinear   5068
5   758.675598      0    RandomBidding   5210
2   398.380849      4       mF_eN_Str1   1465
4   138.547555      6      mF_rfr_Str2   2351
3  1113.428553     11  ConstantBidding  20097


Multiples of budget spent: 15.5
       budget  click        team_name   win
0  553.016002      0       BestLinear  4906
1  413.530429      3    RandomBidding  5042
2  730.308436      4  ConstantBidding  7382
4  692.758241      5       mF_eN_Str1  1426
5  281.790104      6   xgb_lasso_Str2  4840
3   77.646624      9      mF_rfr_Str2  2195




In [18]:
resultsBids.head(20)

,click,payprice,ConstantBidding,RandomBidding,BestLinear,xgb_lasso_Str2,mF_rfr_Str2,mF_eN_Str1
0,0,23,592.5,637.5,695.819299,611.668420,0.000000,0.000000
1,0,75,592.5,667.5,580.693715,560.981722,0.000000,0.000000
2,0,65,592.5,555.0,612.171296,401.078520,0.000000,0.000000
3,0,6,592.5,577.5,580.693715,379.028135,0.000000,0.000000
4,0,5,592.5,540.0,580.693715,518.701773,0.000000,0.000000
5,0,22,592.5,622.5,704.903646,434.899662,0.000000,0.000000
6,0,31,592.5,570.0,612.171296,590.714244,0.000000,0.000000
7,0,20,592.5,607.5,695.819299,673.140355,0.000000,0.000000
8,0,58,592.5,517.5,580.693715,379.028135,0.000000,0.000000
9,0,55,592.5,600.0,621.031217,382.865485,0.000000,0.000000


In [ ]:
dataSubset = resultsBids.sample(frac=0.2, replace=False).copy(True) #divided by 5 to get similar environment
dataSubset.reset_index(drop=True, inplace=True)

In [ ]:
dataSubset.head(10)

# Hold Auction for each

Need a function that will step through the impressions one by one checking who won the impression, whilst keeping track of each player's budget.

In [ ]:
s = Scorer()
df_summary,df = s.set_df(dataSubset, ['ConstantBidding', 'RandomBidding', 'BestLinear','xgb_lasso_Str2', 'mF_rfr_Str2', 'mF_eN_Str1'],6250000,20) 
# /5 because 6 teams instead of 30

# Scores

Summarise the main metrics for each team.

In [ ]:
df_summary['CTR']=df_summary['click']/df_summary['win']
print(df_summary)

In [ ]:
df_summary['win'].sum()/(validation_df.shape[0]*0.2)

In [ ]:
#print(df)